In [1]:
import sys
import os
sys.path.append('/home/glen/2025-LLM-Project')

In [2]:
from src.data_loader import data_load, data_process, data_chunking
from src.vector_db import generate_vector_db, load_vector_db

In [8]:
df = data_load("data/data_list.csv")
df = data_process(df, apply_ocr=False, file_type="all")
all_chunks = data_chunking(df=df, splitter_type="recursive", size=300, overlap=50)
embeddings = generate_vector_db(all_chunks, "nlpai-lab/KoE5")
vector_store = load_vector_db("../data", "nlpai-lab/KoE5")

서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf:  56%|█████▌    | 42/75 [00:01<00:02, 15.52it/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf: 100%|██████████| 49/49 [00:00<00:00, 306.30it/s]
/opt/jhub-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [9]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(all_chunks)
bm25_retriever.k = 3

NameError: name 'retriever' is not defined

In [12]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5],  # Equal weight for hybrid fusion
)

In [16]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever

reranker = ContextualCompressionRetriever(
    base_retriever=hybrid_retriever,
    base_compressor=EmbeddingsFilter(
        embeddings=embeddings,
        similarity_threshold=0.45
    )
)

query = "배드민턴장 및 탁구장 예약방법"
docs = reranker.invoke(query)

# Print top 3 compressed docs
for i, doc in enumerate(docs[:3]):
    print(f"--- 📄 문서 {i+1} ---")
    print(doc.page_content[:500])
    print(doc.metadata)

--- 📄 문서 1 ---
완벽하게 연동 구현 중앙서버 data 정보관리 방식지원이 가능 상세기능은 발주처와 협의후 진행요구사항 번호SFR-007요구사항 분류시스템 기능 요구사항요구사항 명칭예약 시스템 기능 요구사항요구사항상세설명정의예약 시스템 상세 기능 요구사항세부내용 배드민턴장 및 탁구장 예약 시스템을 구분하여 구축 예약기간을 관리자가 설정하면, 해당 기간이 도래했을 때 자동으로예약이 가능하도록 구축 홈페이지에서는 당일예약이 되면 안되며, 현장 키오스크에서는 당일 예약이 가능해야함 예약시 나오는 시간대는 관리자가 직접 설정이 가능해야 함 관리자가
{'사업명': '호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역', '발주 기관': '경기도 안양시', '파일명': '경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.hwp', 'chunk_idx': 45}
--- 📄 문서 2 ---
통해 당일 예약 및 결제 기능을 제공하여 시설이용이 가능 하도록 구현 5) 제공되는 키오스크는 배드민턴과 탁구가 별도로 운영이 가능 하도록 기능 개발4. 기대효과 가. 웹 기반의 예약시스템 구축으로 자동화 및 무인화, 다양한 접근성 제공 등 이용의 편의성 제공과 시설 이용 만족도 향상 나. 데스크 업무 분산을 통한 서비스 품질 강화 및 업무의 효율성 확보 다. 자동화 전산시스템을 통한 비대면 언택트 운영 실현 1. 추진목표 사업추진 방안 호계체육관 배드민턴장 및 탁구장 예약 시스템 구축으로 사용자 편리성 증대, 업무 효율성 증가
{'사업명': '호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역', '발주 기관': '경기도 안양시', '파일명': '경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.hwp', 'chunk_idx': 5}
--- 📄 문서 3 ---
배드민턴장 및 탁구장 예약 시스템 구축(키오스크 서비스) 1) 현장에서 키오스크를 이용하여 인포데스크 방문 없이 예약 및 결제가 가능하도록 관리하는 관리자 페이지 통합 구현 2) 안